In [1]:
# !wget https://huggingface.co/datasets/malaysia-ai/starcoderdata-sample/resolve/main/python.jsonl

In [2]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
from glob import glob

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
ls = []

with open('python.jsonl.1') as fopen:
    for l in tqdm(fopen):
        try:
            l = json.loads(l)
            if len(l['content'].split()) > 5e3:
                continue
            ls.append(l['content'])
            if len(ls) > 200000:
                break
        except:
            pass
len(ls)

200000it [00:11, 17301.50it/s]


198298

In [4]:
client = InferenceClient(
    '', timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [5]:
generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=5120,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [6]:
!mkdir mixtral-python
# !rm -rf mixtral-python/*.json

mkdir: cannot create directory ‘mixtral-python’: File exists


In [7]:
def answer(q, i):
    filename = f'mixtral-python/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
    # for _ in range(5):
        try:
            
            prompt = f"""
Please gain inspiration from the following random code snippet to create a high-quality Python problem. Present your output in two distinct sections: [Problem Description] and [Solution].

Python snippet for inspiration:
```
{q}
```

Guidelines for each section:

1. [Problem Description]: This should be **completely self-contained**, providing all the contextual information one needs to understand and solve the problem. Assume common programming knowledge, but ensure that any specific context, variables, or code snippets pertinent to this problem are explicitly included.

2. [Solution]: Offer a comprehensive, **correct** solution that accurately addresses the [Problem Description] you provided.
""".strip()
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except Exception as e:
            if 'tokens + `max_new_tokens`' in str(e):
                with open(filename, 'w') as fopen:
                    json.dump(False, fopen)
                break
            pass

In [8]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [9]:
from threading import Thread
from queue import Queue

queue = Queue()
questions = ls
urls = [(q, no) for no, q in enumerate(questions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [10]:
answer(*urls[0])

In [11]:
with open('mixtral-python/0.json') as fopen:
    print(json.loads(fopen.read()))

 **[Problem Description]**

The `serializers.py` file provided is part of a Django application that uses the Django Rest Framework (DRF) and the `rest_framework_gis` package to create a GeoJSON API. This application is used to serialize various GeoJSON models related to land use and land cover data.

In this problem, you are required to fix the issues with the `ZonesBaties2018Serializer` class, which is currently missing the `get_couverture` and `get_usage` methods. These methods are essential for generating the correct serialized output as they provide the corresponding labels for the `couverture` and `usage` fields.

**Input:**

The input to the serializer is a model instance of `ZonesBaties2018` class, which has the following fields:

* `id`
* `couverture` (foreign key to `CouvertureSol`)
* `usage` (foreign key to `UsageSol`)
* `surface`

**Output:**

The expected output of the serializer should be a serialized GeoJSON object, which includes the `couverture` and `usage` fields with 

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

  8%|███████▋                                                                                         | 15750/198298 [7:54:04<156:52:23,  3.09s/it]